# Работа с базами данных

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Макрушин С.В. Лекция "Работа с базами данных"
* https://sqliteonline.com/
* https://docs.python.org/3/library/sqlite3.html
* https://www.sqlitetutorial.net/sqlite-create-table/
* https://docs.python.org/3/library/pickle.html
* https://www.geeksforgeeks.org/sql-join-set-1-inner-left-right-and-full-joins/
* https://www.datacamp.com/community/tutorials/group-by-having-clause-sql

## Задачи для совместного разбора

In [48]:
import pandas as pd
import sqlite3, pickle, time
from sqlite3 import Error

In [ ]:
students = pd.DataFrame(
    [
        ("Сотников Евгений Янович", 1),
        ("Степанова Виктория Константиновна", 1),
        ("Горелова Вероника Яновна", 2),
        ("Гришин Иван Романович", 3),
    ],
    columns=["name", "group_id"],
)


groups = list(zip([1, 2, 3], ["ПМ20-1", "ПМ20-2", "ПМ20-3"]))

1. Создайте БД sqlite3 и таблицы Student и StudentGroup в ней.

2. Заполните созданные таблицы данными

3. Выведите на экран фамилии студентов и номера их групп.

## Лабораторная работа 9

### Создание и заполнение базы данных

1\. Создайте файл БД sqlite3 согласно рисунку ниже, на котором определен набор таблиц и связей между ними. Обратите внимание, что поля, выделенные полужирным шрифтом, обозначают первичный ключ таблицы.

Для решения задания напишите скрипт на языке SQL и исполните его при помощи метода `executescript` объекта-курсора.

In [2]:
def create_connection(db_file):
    """ create a database connection to a SQLite database """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
    finally:
        if conn:
            conn.close()

In [3]:
create_connection(".\data\proj.db")

2.6.0


In [4]:
conn = sqlite3.connect(".\data\proj.db")
cursor = conn.cursor()

In [5]:
sql_tag = """
CREATE TABLE IF NOT EXISTS TAG (
    TAG VARCHAR PRIMARY KEY,
    RECIPE_ID INT);
"""

sql_ingredient = """
CREATE TABLE IF NOT EXISTS INGREDIENT (
    INGREDIENT_NAME VARCHAR PRIMARY KEY,
    RECIPE_ID INT);
"""

sql_recipe = """
CREATE TABLE IF NOT EXISTS RECIPE (
    ID INT PRIMARY KEY,
    NAMER VARCHAR,
    MINUTES INT,
    SUBMITTED VARCHAR,
    DESCRIPTION TEXT,
    N_INGREDIENTS INT,
    FOREIGN KEY (ID) REFERENCES TAG(RECIPE_ID),
    FOREIGN KEY (ID) REFERENCES INGREDIENT(RECIPE_ID));
"""

sql_review = """
CREATE TABLE IF NOT EXISTS REVIEW (
    ID INT PRIMARY KEY,
    USER_ID INT,
    RECIPE_ID INT,
    DATER VARCHAR,
    RATING INT,
    REVIEW TEXT,
    FOREIGN KEY (RECIPE_ID) REFERENCES RECIPE(ID));
"""

In [6]:
cursor.executescript(sql_tag)
cursor.executescript(sql_ingredient)
cursor.executescript(sql_recipe)
cursor.executescript(sql_review)

2\. Загрузите данные из файла `recipes_sample.csv` в таблицу `Recipe`. При выполнении задания воспользуйтесь методом `executemany` объекта-курсора.

In [7]:
recipes = pd.read_csv('data/recipes_sample (1).csv', parse_dates=['n_ingredients'])
recipes_sample = recipes[['id', 'name', 'minutes','submitted','description','n_ingredients']]
int_ingredients = [int(str(ingred[:-2])) if str(ingred) != 'nan' else 0 for ingred in recipes_sample['n_ingredients']]
recipes_sample['n_ingredients'] = pd.Series(int_ingredients)
recipes_sample

C:\Users\admin\AppData\Local\Temp/ipykernel_19612/3634523435.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recipes_sample['n_ingredients'] = pd.Series(int_ingredients)


,id,name,minutes,submitted,description,n_ingredients
0,44123,george s at the cove black bean soup,90,2002-10-25,an original recipe created by chef scott meska...,18
1,67664,healthy for them yogurt popsicles,10,2003-07-26,my children and their friends ask for my homem...,0
2,38798,i can t believe it s spinach,30,2002-08-29,"these were so go, it surprised even me.",8
3,35173,italian gut busters,45,2002-07-27,my sister-in-law made these for us at a family...,0
4,84797,love is in the air beef fondue sauces,25,2004-02-23,i think a fondue is a very romantic casual din...,0
...,...,...,...,...,...,...
29995,267661,zurie s holey rustic olive and cheddar bread,80,2007-11-25,this is based on a french recipe but i changed...,10
29996,386977,zwetschgenkuchen bavarian plum cake,240,2009-08-24,"this is a traditional fresh plum cake, thought...",11
29997,103312,zwiebelkuchen southwest german onion cake,75,2004-11-03,this is a traditional late summer early fall s...,0
29998,486161,zydeco soup,60,2012-08-29,this is a delicious soup that i originally fou...,0


In [8]:
recipes_sample.dtypes

id                int64
name             object
minutes           int64
submitted        object
description      object
n_ingredients     int64
dtype: object

In [9]:
sql = '''
INSERT INTO Recipe(id, namer, minutes, submitted, description, n_ingredients) VALUES (?, ?, ?, ?, ?, ?)
'''
cursor.executemany(sql, recipes_sample.values.tolist())
conn.commit()
sql = f'''SELECT * FROM RECIPE;'''
pd.DataFrame(cursor.execute(sql).fetchmany(3), columns=list(('id', 'namer', 'minutes', 'submitted', 'description', 'n_ingredients')))

,id,namer,minutes,submitted,description,n_ingredients
0,44123,george s at the cove black bean soup,90,2002-10-25,an original recipe created by chef scott meska...,18
1,67664,healthy for them yogurt popsicles,10,2003-07-26,my children and their friends ask for my homem...,0
2,38798,i can t believe it s spinach,30,2002-08-29,"these were so go, it surprised even me.",8


3\. Загрузите данные из файла `tags_sample.pickle` в таблицу `Tag`. При выполнении задания воспользуйтесь методом `executemany` объекта-курсора. Для считывания файла с данными воспользуйтесь пакетом `pickle`. Обратите внимание, что перед добавлением записей в базу данные нужно привести к соответствующему таблице в БД виду (в каждой строчке столбца tag должен находиться один тэг).

In [10]:
with open('data/tags_sample.pickle', 'rb') as file:
    tags_sample = pickle.load(file)
tags_sample = pd.DataFrame(tags_sample)
tags_sample = tags_sample.explode('tag')[['tag','id']]
tags_sample

,tag,id
0,eggs-dairy,48
0,oven,48
0,main-ingredient,48
0,time-to-make,48
0,course,48
...,...,...
29983,course,536747
29983,desserts,536747
29983,for-large-groups,536747
29983,preparation,536747


In [11]:
sql = '''
INSERT or IGNORE INTO Tag(tag, recipe_id) VALUES (?, ?)
'''
cursor.executemany(sql, tags_sample.values.tolist())
conn.commit()
sql = '''SELECT * FROM Tag;'''
pd.DataFrame(cursor.execute(sql).fetchmany(5), columns=['tag', 'recipe_id'])

,tag,recipe_id
0,eggs-dairy,48
1,oven,48
2,main-ingredient,48
3,time-to-make,48
4,course,48


4\. Загрузите данные из файла `reviews_sample.csv` в таблицу `Review`. При выполнении задания воспользуйтесь методом `pd.DataFrame.to_sql`.

In [12]:
reviews = pd.read_csv('data/reviews_sample.csv')
reviews.rename(columns = {'Unnamed: 0' : 'id'}, inplace = True)
reviews

,id,user_id,recipe_id,date,rating,review
0,370476,21752,57993,2003-05-01,5,Last week whole sides of frozen salmon fillet ...
1,624300,431813,142201,2007-09-16,5,So simple and so tasty! I used a yellow capsi...
2,187037,400708,252013,2008-01-10,4,"Very nice breakfast HH, easy to make and yummy..."
3,706134,2001852463,404716,2017-12-11,5,These are a favorite for the holidays and so e...
4,312179,95810,129396,2008-03-14,5,Excellent soup! The tomato flavor is just gre...
...,...,...,...,...,...,...
126691,1013457,1270706,335534,2009-05-17,4,This recipe was great! I made it last night. I...
126692,158736,2282344,8701,2012-06-03,0,This recipe is outstanding. I followed the rec...
126693,1059834,689540,222001,2008-04-08,5,"Well, we were not a crowd but it was a fabulou..."
126694,453285,2000242659,354979,2015-06-02,5,I have been a steak eater and dedicated BBQ gr...


In [13]:
sql = '''
INSERT INTO Review(id, user_id, recipe_id, dater, rating, review) VALUES (?, ?, ?, ?, ?, ?)
'''
cursor.executemany(sql, reviews.values.tolist())
conn.commit()
sql = '''SELECT * FROM review;'''
pd.DataFrame(cursor.execute(sql).fetchmany(3), columns=['id', 'user_id', 'recipe_id', 'date', 'rating', 'review'])

,id,user_id,recipe_id,date,rating,review
0,370476,21752,57993,2003-05-01,5,Last week whole sides of frozen salmon fillet ...
1,624300,431813,142201,2007-09-16,5,So simple and so tasty! I used a yellow capsi...
2,187037,400708,252013,2008-01-10,4,"Very nice breakfast HH, easy to make and yummy..."


In [14]:
cursor.execute(sql).fetchmany(1)

[(370476,
  21752,
  57993,
  '2003-05-01',
  5,
  "Last week whole sides of frozen salmon fillet was on sale in my local supermarket, so I bought tons (okay, only 3, but total weight was over 10 pounds).  This recipe is perfect for salmon fillet, even though it calls for salmon steaks.  I cut up the salmon into individual portions and followed the instructions exactly.  I'm on one of those food combining diets, so I left out the white wine but added just a dash of white wine vinegar instead (just a little bit, not enough to change the taste of the dish).  Super yummy, and leftovers for lunch today (lucky me)!")]

5\. Загрузите данные из файла `ingredients_sample.csv` в таблицу `Ingredients`. При выполнении задания воспользуйтесь методом `DataFrame.to_sql`.

Обратите внимание, перед вызовом метода `to_sql` вам требуется привести фрейм к соответствующему таблице в БД виду.

In [15]:
ingredients = pd.read_csv('data/ingredients_sample.csv')
ingredients['ingredients'] = ingredients['ingredients'].str.split(';').tolist()
ingredients = ingredients.explode('ingredients')[['ingredients','recipe']]
ingredients

,ingredients,recipe
0,cornstarch,48
0,milk,48
0,eggs,48
0,margarine,48
0,butter,48
...,...,...
29999,flaked sea salt,536747
29999,gelatin powder,536747
29999,powdered sugar icing,536747
29999,lemon curd,536747


In [17]:
ingredients.to_sql("ingredients", conn, if_exists="append", index=False)
conn.commit()
sql = '''SELECT * FROM ingredients'''
pd.DataFrame(cursor.execute(sql).fetchmany(3), columns=['ingredient_name', 'recipe_id'])

,ingredient_name,recipe_id
0,cornstarch,48
1,milk,48
2,eggs,48


### Получение данных из базы

6\. Напишите и выполните запрос на языке SQL, который считает кол-во рецептов, опубликованных в 2010 году и имеющих длину не менее 15 минут. Для выполнения запроса используйте метод `execute` объекта-курсора. Выведите искомое количество на экран.

In [18]:
sql = '''
SELECT count(id)
FROM recipe
where submitted like '2010%' and minutes>=15
'''
cursor.execute(sql).fetchall()[0][0]

1319

7\. Напишите и выполните запрос на языке SQL, который возращает id рецептов, не имеющих ни одного отзыва отзывов с рейтингом, меньше 4. Для выполнения запроса используйте функцию `pd.read_sql_query`. Выведите полученный результат на экран.

In [19]:
sql = '''
SELECT distinct recipe_id
FROM review
where rating>=4
order by recipe_id
'''
pd.read_sql_query(sql, conn)

,RECIPE_ID
0,55
1,66
2,91
3,94
4,128
...,...
26313,536360
26314,536473
26315,536547
26316,536728


8\. Создайте `pd.DataFrame`, содержащий данные из таблицы `Tag`. Создайте `pd.DataFrame`, содержащий данные из таблицы `Recipe` (для создания фреймов можно воспользоваться функцией `read_sql_query`). 

Используя механизмы группировки и объединения, которые предоставляет `pandas`, выведите на экран названия и количество тегов 5 рецептов, которые имеют наибольшее количество тэгов. Измерьте время выполнения работы вашего кода (в замеры включите время, которое тратится на загрузку таблиц).

In [50]:
start = time.time()

sql = '''
SELECT *
FROM Tag
'''
tag = pd.read_sql_query(sql, conn)

sql = '''
SELECT *
FROM Recipe
'''
Recipe = pd.read_sql_query(sql, conn)

rec = Recipe.merge(tag, left_on='ID', right_on='RECIPE_ID')
top5 = pd.DataFrame(rec.value_counts('ID')[:5].index.tolist(), columns = ['id'])
top5['count_tags'] = rec.value_counts('ID')[:5].values.tolist()
print(top5.merge(Recipe,left_on = 'id', right_on ='ID')[['NAMER','count_tags']])

end = time.time()
print()
print(end - start,'секунд')

                                        NAMER  count_tags
0  betty crocker s southwestern guacamole dip          17
1                            boston cream pie          16
2                                   bugwiches          10
3                 black coffee barbecue sauce           9
4         b c  cherry and raspberry preserves           9

0.2241518497467041 секунд


In [21]:
tag

,TAG,RECIPE_ID
0,eggs-dairy,48
1,oven,48
2,main-ingredient,48
3,time-to-make,48
4,course,48
...,...,...
499,hidden-valley-ranch,494156
500,prepared-potatoes,496522
501,laotian,502817
502,a1-sauce,518045


In [22]:
Recipe

,ID,NAMER,MINUTES,SUBMITTED,DESCRIPTION,N_INGREDIENTS
0,44123,george s at the cove black bean soup,90,2002-10-25,an original recipe created by chef scott meska...,18
1,67664,healthy for them yogurt popsicles,10,2003-07-26,my children and their friends ask for my homem...,0
2,38798,i can t believe it s spinach,30,2002-08-29,"these were so go, it surprised even me.",8
3,35173,italian gut busters,45,2002-07-27,my sister-in-law made these for us at a family...,0
4,84797,love is in the air beef fondue sauces,25,2004-02-23,i think a fondue is a very romantic casual din...,0
...,...,...,...,...,...,...
29995,267661,zurie s holey rustic olive and cheddar bread,80,2007-11-25,this is based on a french recipe but i changed...,10
29996,386977,zwetschgenkuchen bavarian plum cake,240,2009-08-24,"this is a traditional fresh plum cake, thought...",11
29997,103312,zwiebelkuchen southwest german onion cake,75,2004-11-03,this is a traditional late summer early fall s...,0
29998,486161,zydeco soup,60,2012-08-29,this is a delicious soup that i originally fou...,0


9\. Используя механизмы группировки и объединения, которые предоставляет SQL, выведите на экран названия и количество тегов 5 рецептов, которые имеют наибольшее количество тэгов. При выполнении задания воспользуйтесь методом `execute` объекта-курсора. Измерьте время выполнения работы вашего кода.

Вся необходимая логика (группировки, объединения, выбор топ-5 строк) должна быть реализована на SQL, а не в виде кода на Python.

In [53]:
start = time.time()

sql = '''
SELECT recipe.namer, count(recipe.namer)
FROM recipe join Tag on recipe.id = Tag.recipe_id
GROUP BY recipe.namer
ORDER BY COUNT('recipe.namer') DESC
LIMIT 5;
'''
print(*cursor.execute(sql).fetchall(), sep='\n')

end = time.time()
print()
print(end - start, 'секунд')

('betty crocker s southwestern guacamole dip', 17)
('boston cream pie', 16)
('bugwiches', 10)
('chicken with portabella mushrooms', 9)
('black coffee barbecue sauce', 9)

0.005995512008666992 секунд


10\. Запросите у пользователя id рецепта и верните информацию об этом рецепте. Если рецепт отсутствует, выведите соответствующее сообщение. Для подстановки значения id необходимо воспользоваться специальным синтаксисом, которые предоставляет `sqlite` для этих целей.Продемонстрируйте работоспособность вашего решения.

In [29]:
recipe_id = input('Введите id: ')

sql = '''
select *
from Recipe
where id=?
'''

r = cursor.execute(sql, (recipe_id,))
result = r.fetchall()
if result:
  print(result)
else:
  print(f'id {recipe_id} не найден')

[(44123, 'george s at the cove  black bean soup', 90, '2002-10-25', "an original recipe created by chef scott meskan, george's at the cove. we enjoyed this when we visited this restaurant in la jolla, california. this recipe is requested so often, they have it printed and ready at the hostess stand. it's unbeatable at the restaurant, but i do a pretty good job at home, too, if i do say so myself!", 18)]
